In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer 
import numpy as np
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split

In [2]:
path =r'F:\Study\NU\Data Mining\Project\Data' # use your path
allFiles = glob.glob(os.path.join(path, "*.txt"))
df = pd.concat((pd.read_csv(f, delimiter="\t", header=None) for f in allFiles))


In [3]:
df = df.rename(index=str, columns={0: "id", 1: "label", 2:"text"})

In [37]:
df_train,df_test,train_labels,test_labels = train_test_split(df["text"],                   
                                                 df['label'], test_size=0.20,
                                                 random_state=1)

In [13]:
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

In [38]:
vectorizer = CountVectorizer().fit(df_train)
tweet_train_vectorized = vectorizer.transform( df_train)
tweet_test_vectorized = vectorizer.transform( df_test)
print (len(vectorizer.get_feature_names()))


28745


In [39]:
clfr = svm.SVC(kernel='linear', C=1)
clfr.fit(tweet_train_vectorized,train_labels)
predicted = clfr.predict(tweet_test_vectorized)
acc = metrics.accuracy_score(test_labels,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(test_labels,predicted))

accuracy = 62.2312246806%
             precision    recall  f1-score   support

   negative       0.48      0.45      0.46       498
    neutral       0.62      0.63      0.63      1361
   positive       0.67      0.68      0.67      1350

avg / total       0.62      0.62      0.62      3209



In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [32]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(tweet_train_vectorized,train_labels)
predicted = clf.predict(tweet_test_vectorized)
acc = metrics.accuracy_score(test_labels,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(test_labels,predicted))


accuracy = 62.6986600187%
             precision    recall  f1-score   support

   negative       0.47      0.45      0.46       498
    neutral       0.63      0.63      0.63      1361
   positive       0.68      0.69      0.68      1350

avg / total       0.63      0.63      0.63      3209



In [33]:
clf.best_estimator_

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
text_file = open("F:\\Study\\NU\\Data Mining\\Project\\Resources\\stopwords.txt", "r")
lines = text_file.read().split('\n')

In [27]:
def preprocessing(tweet):
    return ' '.join(list(string for string in tweet.split(" ") if (string not in lines)))

Now I am trying to count negatvie and postive word apps within each tweet

In [70]:
df['p_count'] = 0
df['n_count'] = 0

In [35]:
df['text'] = df['text'].apply(preprocessing)

In [44]:
text_file = open("F:\\Study\\NU\\Data Mining\\Project\\Resources\\opinion-lexicon-English-Bing_liu\\positive-words.txt", "r")
positive_words = text_file.read().split('\n')

text_file = open("F:\\Study\\NU\\Data Mining\\Project\\Resources\\opinion-lexicon-English-Bing_liu\\negative-words.txt", "r")
negative_words = text_file.read().split('\n')

In [69]:
def p_counting(tweet):
    positive_counter = tweet.p_count
    for string in tweet.text.split(" "):
        if string in positive_words:
            positive_counter +=1
    return positive_counter

def n_counting(tweet):
    negative_counter = tweet.n_count
    for string in tweet.text.split(" "):
        if string in negative_words:
            negative_counter +=1
    return negative_counter

In [71]:
df['p_count'] = df.apply(p_counting, axis=1)
df['n_count'] = df.apply(n_counting, axis=1)